# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# To dispay map 
# !jupyter nbextension enable --py gmaps


In [2]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import os
from pprint import pprint
from scipy.stats import linregress
from api_keys import weather_api_key
from api_keys import google_api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load city data exported from WeatherPy Part I
weather_data_to_load = "../output_data/cities_part_I.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,315,alofi,58,NU,1602430333,100,-19.06,-169.92,22.00,6.20
1,497,eureka,1,US,1602430632,100,40.80,-124.16,10.00,1.50
2,444,uyaan,100,PH,1602430780,100,7.86,124.04,17.98,1.35
3,213,ulladulla,75,AU,1602430730,100,-35.35,150.47,14.44,2.60
4,224,aykhal,90,RU,1602430733,100,66.00,111.50,-9.00,1.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# location coordinates
coordinates = weather_data[["Lat", "Lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)


# Access maps with unique API key
gmaps.configure(api_key=google_api_key)

# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '500px',
    'border': '2px solid black',
    'padding': '2px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


heatmap_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='2px solid black', height='500px', margin='0 auto 0 auto', padding='2px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal

selection_df = pd.DataFrame(weather_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

# This data set temperature is in Celcius
ideal_choices_df = selection_df[(selection_df["Max Temp"] <= 27) & (selection_df["Max Temp"] > 20)\
                                & (selection_df["Wind Speed"] < 10)\
                                & (selection_df["Cloudiness"] == 0)]

ideal_choices_df


,City,Max Temp,Wind Speed,Cloudiness
204,rikitea,22.90,6.41,0
222,nhulunbuy,24.00,2.10,0
223,salalah,27.00,2.60,0
232,hobyo,26.96,4.80,0
268,bandarbeyla,26.48,2.69,0
356,torres,20.59,6.74,0
364,qadian,25.00,1.00,0
396,karratha,23.99,5.86,0
397,ourem,25.00,7.70,0
408,ilhabela,23.07,3.16,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_list = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": google_api_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_list.append(response['results'][0]['name'])
    except:
        hotel_list.append("")
        
hotel_df["Hotel Name"] = hotel_list
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()



,City,Country,Lat,Lng,Hotel Name
0,alofi,NU,-19.06,-169.92,Alofi
1,eureka,US,40.80,-124.16,Eureka
2,uyaan,PH,7.86,124.04,Ganassi
3,ulladulla,AU,-35.35,150.47,Ulladulla
4,aykhal,RU,66.00,111.50,Aikhal


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='2px solid black', height='500px', margin='0 auto 0 auto', padding='2px', wi…